# Extraction of Clinical Trials from Clinicaltrials.gov sorted by start date

In [2]:
# Main loop: Fetch and filter trials from the API
import json
import os
import requests
from datetime import datetime, timedelta
from time import sleep
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY")

# Config
condition = "Pulmonary Arterial Hypertension"
cutoff_date = (datetime.today() - timedelta(days=365 * 10)).date()
url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.cond": condition,
    "query.term":"INTERVENTIONAL",
    "pageSize": 100,
    "format": "json"
}
MAX_RETRIES = 3
RETRY_DELAY = 2
filtered = []
filtered_filings = []
next_page_token = None
studies = []
print("Fetching data from ClinicalTrials.gov with pagination...\n")
while True:
    for attempt in range(MAX_RETRIES):
        try:
            if next_page_token:
                params["pageToken"] = next_page_token
            else:
                params.pop("pageToken", None)
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            studies.extend(data.get("studies", []))
            next_page_token = data.get("nextPageToken")
            break
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"Retrying due to error: {e}")
                sleep(RETRY_DELAY)
            else:
                print("Failed to fetch after retries:", e)
                next_page_token = None
    if not next_page_token:
        break
    
def get_start_date(study):
    for fmt in ("%Y-%m-%d", "%Y-%m"):
        try:
            date_str = study["protocolSection"]["statusModule"]["startDateStruct"]["date"]
            return datetime.strptime(date_str, fmt)
        except (KeyError, ValueError):
            continue
        # fallback if date missing or malformed
    return datetime.min

sorted_studies = sorted(studies, key=get_start_date, reverse=True)

def parse_start_date(date_str):
        for fmt in ("%Y-%m-%d", "%Y-%m"):
            try:
                return datetime.strptime(date_str, fmt).date()
            except ValueError:
                continue
        return None  # or raise an error if preferred
    


Fetching data from ClinicalTrials.gov with pagination...



# Find out the studies where the sponsor = public(Using the SEC API), company = industry and start date is within last 10 years

In [4]:
import requests
import json
import os
from datetime import datetime, timedelta


for s in sorted_studies:
    date_str = s["protocolSection"]["statusModule"]["startDateStruct"]["date"]
    start_date = parse_start_date(date_str)
    isCondition = False
    for c in s["protocolSection"]["conditionsModule"]["conditions"]:
        if c == condition:
            isCondition = True
            print("Condition found:", c)
            break
    #Check if the lead sponsor is an industry and the start date is within the last 10 years
    if s["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["class"] == "INDUSTRY" and start_date>= cutoff_date and isCondition:
        print(s["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"])
        company_name = s["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"]

        # Endpoint
        edgar_url = "https://api.sec-api.io"
        
        # Query payload (JSON body)
        payload = {
            "query": f'companyName:"{company_name}" AND (formType:"10-K" OR formType:"8-K")',
            "from": "0",
            "size": "50",
            "sort": [{ "filedAt": { "order": "desc" }}]
        }
        print("payload",payload)
        
        # Token as query param
        params_edgar = {
            "token": API_KEY
        }
        
        # Send POST request
        response_edgar = requests.post(edgar_url, params=params_edgar, json=payload)
        data_edgar = response_edgar.json()
        isPublic = data_edgar.get("filings",[])
        if isPublic:
            # Sanitize the company name for filename
            filtered_filings.append(data_edgar)
            safe_company_name = "".join(c if c.isalnum() or c in (" ", "_", "-") else "_" for c in company_name).strip().replace(" ", "_")

            # Set up file path (in a folder called 'edgar_results')
            file_path = f"edgar_results/{safe_company_name}.json"
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            

            # Save the EDGAR data to a JSON file
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data_edgar, f, ensure_ascii=False, indent=2)

            print(f"Saved EDGAR data for {company_name} → {file_path}")
            filtered.append(s)

    if len(filtered)==5:
        break
for trials in filtered:
    print(trials["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"])

Condition found: Pulmonary Arterial Hypertension
Guangzhou Magpie Pharmaceuticals Co., Ltd.
payload {'query': 'companyName:"Guangzhou Magpie Pharmaceuticals Co., Ltd." AND (formType:"10-K" OR formType:"8-K")', 'from': '0', 'size': '50', 'sort': [{'filedAt': {'order': 'desc'}}]}
Condition found: Pulmonary Arterial Hypertension
Merck Sharp & Dohme LLC
payload {'query': 'companyName:"Merck Sharp & Dohme LLC" AND (formType:"10-K" OR formType:"8-K")', 'from': '0', 'size': '50', 'sort': [{'filedAt': {'order': 'desc'}}]}
Condition found: Pulmonary Arterial Hypertension
Condition found: Pulmonary Arterial Hypertension
Condition found: Pulmonary Arterial Hypertension
Apollo Therapeutics Ltd
payload {'query': 'companyName:"Apollo Therapeutics Ltd" AND (formType:"10-K" OR formType:"8-K")', 'from': '0', 'size': '50', 'sort': [{'filedAt': {'order': 'desc'}}]}
Condition found: Pulmonary Arterial Hypertension
Condition found: Pulmonary Arterial Hypertension
Condition found: Pulmonary Arterial Hyperte

# Storing the filtered data of 5 most recent trials into a folder

In [6]:
for t in filtered:
    print(t["protocolSection"]["identificationModule"]["briefTitle"])
    company_name = t["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"]
    safe_company_name = "".join(c if c.isalnum() or c in (" ", "_", "-") else "_" for c in company_name).strip().replace(" ", "_")
    nctid = t["protocolSection"]["identificationModule"]["nctId"]
    file_path_2 = f"trail_results/{safe_company_name}/{nctid}.json"
    os.makedirs(os.path.dirname(file_path_2), exist_ok=True)

    # Save the EDGAR data to a JSON file
    with open(file_path_2, "w", encoding="utf-8") as f:
        json.dump(t, f, ensure_ascii=False, indent=2)

A Study to Learn About the Study Medicine Called PF-07868489 in Healthy Adult People and in People With Pulmonary Arterial Hypertension
A Study to Investigate the Safety and Efficacy of KER-012 in Combination With Background Therapy in Adult Participants With Pulmonary Arterial Hypertension (PAH) (TROPOS Study).
Inhaled Imatinib Pulmonary Arterial Hypertension Clinical Trial - Follow Up Long Term Extension (IMPAHCT-FUL)
A Study of a Mean Pulmonary Artery Pressure-Targeted Approach With Early and Rapid Treprostinil Therapy to Reverse Right Ventricular Remodeling in Participants With Pulmonary Arterial Hypertension
A Study of AV-101 (Dry Powder Inhaled Imatinib) in Patients With Pulmonary Arterial Hypertension (PAH)


# Searching the 8K and 10K filings specific to the given trial using the keywords specific to the trial and storing the urls of all 8K and 10K into all_results array. Used the SEC API to find the urls of the 8K and 10K filings.

In [8]:
import json
from bs4 import BeautifulSoup
import requests
import re
import os
from datetime import datetime
data = []
for f1, f2 in zip(filtered, filtered_filings):
    data.append({
        "trial": f1,
        "filing": f2
    })
all_results = []
for item in data:
    trial_data = item["trial"]
    filing_data = item["filing"]
    print()
    print("Trial Company:", trial_data["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"])
    
    keywords = [condition]
    # Extract and sanitize drug name
    import re
    def simplify_drug_name(label):
        # Lowercase and remove dosage
        label = label.lower()
        # Remove special characters
        label = re.sub(r'[^a-z0-9\s-]', '', label)
        words = label.split()
        # Keep only the first few "descriptive" words
        filtered = [w for w in words]
        return filtered
    
    # Extract drug names from the trial data
    for drug in trial_data["protocolSection"]["armsInterventionsModule"]["interventions"]:
        keyword = simplify_drug_name(drug["name"])
        for k in keyword:
            keywords.append(k)

    print("Keywords:", keywords)
    # Find 8K URL
    filings_8k = []
    for filings in filing_data.get("filings",[]):
        event_date = filings.get("filedAt")
        for doc in filings.get("documentFormatFiles", []):
            if doc.get("description", "").strip().upper() == "8-K" or doc.get("description", "").strip().upper() == "FORM 8-K":
                url = doc.get("documentUrl")
                if url and "/ix?doc=" in url:
                    # Strip off the IX viewer part
                    url = url.split("doc=")[-1]
                    url = "https://www.sec.gov" + url if url.startswith("/") else url
                filings_8k.append({url: event_date})
    print("8-K Filings:", filings_8k)
    
    # Find 10K URL
    filings_10k = []
    for filings in filing_data.get("filings",[]):
        event_date = filings.get("filedAt")
        for doc in filings.get("documentFormatFiles", []):
            if doc.get("description", "").strip().upper() == "10-K" or doc.get("description", "").strip().upper() == "FORM 10-K":
                url = doc.get("documentUrl")
                if url and "/ix?doc=" in url:
                    # Strip off the IX viewer part
                    url = url.split("doc=")[-1]
                    url = "https://www.sec.gov" + url if url.startswith("/") else url
                filings_10k.append({url: event_date})
    print("10-K Filings:", filings_10k)
    
    
    result_8K = []
    result_10K = []
        
    if filings_8k and keywords:
        for f in filings_8k:
            filing = list(f.keys())[0]
            event_date = f[filing]
            response = requests.get(filing, headers={"User-Agent": "jeedigunta.s@northeastern.edu"})
            if response.ok:
                soup = BeautifulSoup(response.text, "html.parser")
                visible_text = soup.get_text(separator=" ").lower()
                for keyword in keywords:
                    if keyword in visible_text and len(keyword) >= 4:
                        print(f"Match found for drug '{keyword}' in 8-K")
                        # Extract the event date
                        result_8K.append({filing: event_date})
                    else:
                        print(f"No match for drug '{keyword}' in 8-K")
            else:
                print("Failed to fetch 8-K content")
    else:
        print("8-K document or drug name not found.")
    print("Results:", result_8K)
    
    if filings_10k and keywords:
        for f in filings_10k:
            filing = list(f.keys())[0]
            event_date = f[filing]
            response = requests.get(filing, headers={"User-Agent": "jeedigunta.s@northeastern.edu"})
            if response.ok:
                soup = BeautifulSoup(response.text, "html.parser")
                visible_text = soup.get_text(separator=" ").lower()
                for keyword in keywords:
                    if keyword in visible_text and len(keyword) >= 4:
                        print(f"Match found for drug '{keyword}' in 10-K")
                        if event_date:
                            result_10K.append({filing: event_date})
                    else:
                        print(f"No match for drug '{keyword}' in 10-K")
            else:
                print("Failed to fetch 10-K content")
    else:
        print("10-K document or drug name not found.")
    print("Results:", result_10K)

    all_results.append({
        "trial": trial_data["protocolSection"]["identificationModule"]["nctId"],
        "results-8K": result_8K,
        "results-10K": result_10K
    })
    print("All Results:", all_results)


Trial Company: Pfizer
Keywords: ['Pulmonary Arterial Hypertension', 'pf-07868489', 'placebo', 'for', 'pf-07868489']
8-K Filings: [{'https://www.sec.gov/Archives/edgar/data/78003/000007800325000111/pfe-20250429.htm': '2025-04-29T08:02:49-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800325000109/pfe-20250424.htm': '2025-04-28T16:24:01-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800325000017/pfe-20250204.htm': '2025-02-04T08:11:20-05:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800324000188/pfe-20241029.htm': '2024-10-29T08:06:28-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800324000155/pfe-20240730.htm': '2024-07-30T08:04:27-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800324000130/pfe-20240709.htm': '2024-07-10T16:11:00-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800324000103/pfe-20240501.htm': '2024-05-01T08:11:26-04:00'}, {'https://www.sec.gov/Archives/edgar/data/78003/000007800

# Use the all_results array to get the 4(atmost) most recent 8K and 10K filings and store it as pdfs using SEC PDF download API

In [10]:
import os
import requests
from datetime import datetime

# === CONFIG ===
SEC_API_TOKEN = API_KEY
SEC_API_ENDPOINT = "https://api.sec-api.io/filing-reader"
NUM_FILINGS = 4  # change to 4 for top 4 filings

# === HELPERS ===
def parse_iso_date(date_str):
    try:
        return datetime.fromisoformat(date_str)
    except Exception:
        return datetime.min

def download_pdf(filing_url, save_path):
    params = {
        "token": SEC_API_TOKEN,
        "url": filing_url
    }
    try:
        response = requests.get(SEC_API_ENDPOINT, params=params)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"Saved: {save_path}")
            return True
        else:
            print(f"Failed ({response.status_code}): {filing_url}")
    except Exception as e:
        print(f"Exception: {filing_url} — {e}")
    return False

# === MAIN LOGIC ===
def main():

    output_root = "forms"
    os.makedirs(output_root, exist_ok=True)

    for trial_entry in all_results:
        trial_name = trial_entry["trial"].replace(" ", "_").replace(",", "").replace(".", "")
        trial_folder = os.path.join(output_root, trial_name)
        os.makedirs(trial_folder, exist_ok=True)

        for report_key in ["results-8K", "results-10K"]:
            folder_name = "8K" if "8K" in report_key else "10K"
            folder_path = os.path.join(trial_folder, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            filings = []
            for entry in trial_entry.get(report_key, []):
                for url, dt in entry.items():
                    filings.append((url, parse_iso_date(dt)))

            # Sort and deduplicate
            seen = set()
            sorted_unique = []
            for url, dt in sorted(filings, key=lambda x: x[1], reverse=True):
                if url not in seen:
                    sorted_unique.append((url, dt))
                    seen.add(url)
                if len(sorted_unique) >= NUM_FILINGS:
                    break

            for url, dt in sorted_unique:
                filename = os.path.basename(url).replace("/", "_") + ".pdf"
                save_path = os.path.join(folder_path, filename)
                print(f"⬇️ Downloading {folder_name} for {trial_name} → {filename}")
                download_pdf(url, save_path)

    print("All latest filings downloaded.")

main()
# === END OF SCRIPT ===


⬇️ Downloading 10K for NCT06137742 → pfe-20211231.htm.pdf
Saved: forms/NCT06137742/10K/pfe-20211231.htm.pdf
⬇️ Downloading 8K for NCT05975905 → kros-20250115.htm.pdf
Saved: forms/NCT05975905/8K/kros-20250115.htm.pdf
⬇️ Downloading 8K for NCT05975905 → kros-20241212.htm.pdf
Saved: forms/NCT05975905/8K/kros-20241212.htm.pdf
⬇️ Downloading 8K for NCT05975905 → kros-20240930.htm.pdf
Saved: forms/NCT05975905/8K/kros-20240930.htm.pdf
⬇️ Downloading 8K for NCT05975905 → kros-20240903.htm.pdf
Saved: forms/NCT05975905/8K/kros-20240903.htm.pdf
⬇️ Downloading 10K for NCT05975905 → kros-20241231.htm.pdf
Saved: forms/NCT05975905/10K/kros-20241231.htm.pdf
⬇️ Downloading 10K for NCT05975905 → kros-20231231.htm.pdf
Saved: forms/NCT05975905/10K/kros-20231231.htm.pdf
⬇️ Downloading 8K for NCT05557942 → tmb-20240625x8k.htm.pdf
Saved: forms/NCT05557942/8K/tmb-20240625x8k.htm.pdf
⬇️ Downloading 8K for NCT05557942 → tm2417394d1_8k.htm.pdf
Saved: forms/NCT05557942/8K/tm2417394d1_8k.htm.pdf
⬇️ Downloading 8K 

# Used OpenAI API to create a function which extracts the drug information in the specified format

In [12]:
import openai
import os
import json
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_drug_metadata_from_description(drug_data):
    results =[]
    def extract_data_for_each_drug(drug):
        description = drug.get("description", "")
        raw_name = drug.get("name", "")
        synonyms = ", ".join(drug.get("otherNames", []))
    
        prompt = f"""
    You are a biomedical assistant. Extract the following structured metadata about an interventional drug
    based on its name, description, and known synonyms. Be concise and infer dosage/frequency/formulation if clearly mentioned.
    
    Respond ONLY in this JSON format:
    
      {{
        "name": string,
        "dose": string or null,
        "frequency": string or null,
        "formulation": string or null
      }}
    
    Input:
    - Name: {raw_name}
    - Synonyms: {synonyms}
    - Description: "{description}"
    """
    
        try:
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=300,
            )
            return json.loads(response.choices[0].message.content.strip())
        except Exception as e:
            print(f"⚠️ GPT extraction error: {e}")
            return {
                "name": raw_name,
                "dose": None,
                "frequency": None,
                "formulation": None
            }
    for drug in drug_data:
        results.append(extract_data_for_each_drug(drug))
    return results

def extract_endpoints_from_outcome_module(outcome_module):
    """
    Calls GPT to extract endpoint information in strict JSON format.
    Will only return fields it can confidently extract (no hallucination).
    """
    prompt = f"""
    You are a biomedical data analyst. Given the following outcome module data from a clinical trial,
    extract all endpoints in this precise JSON format:
    
     Respond **only** in the following JSON format:
    
      [
        {{
          "name": "",
          "description": "",
          "timepoint": "",
          "arm": "intervention",
          "average_value": null,
          "upper_end": null,
          "lower_end": null,
          "statistical_significance": ""
        }}
      ]
    
    🧠 STRICT RULES:
    - There can be mutiple endpoints, that is why endpoints is an array
    - If you cannot confidently find a value, leave it as null or "".
    - Do not guess or hallucinate.
    - Only include real arms and values mentioned in the input.
    - For each arm, summarize the result for that outcome (if possible).
    - arm can be either placebo or intervention- it should be intervention if placebo is not mentioned
    - Do not give any simple text in response. Only give the above JSON Array.
    
    Input:
    ```json
    {json.dumps(outcome_module, indent=2)}

    If not mentioned anywhere, return null for those fields. But please return some output for each of the measure I am giving you. Respond **only** in the following JSON format:
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=800,
        )
        content = response.choices[0].message.content.strip()
        print(content)
        return json.loads(content)
    except Exception as e:
        print(f"❌ GPT Extraction Error: {e}")
        return {"endpoints": []}
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch

load_dotenv()
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")

def search_google_serpapi(query, num_results=5):
    if not SERP_API_KEY:
        raise ValueError("Missing SERPAPI_API_KEY in environment variables")

    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    links = []
    for res in results.get("organic_results", []):
        links.append({
            "title": res.get("title"),
            "link": res.get("link"),
            "snippet": res.get("snippet", "")
        })

    return links

def fetch_endpoint_measurements_with_context(outcomes_module, trial_name, condition, drug):
    """
    Uses GPT to extract endpoint data with measurement values (if available from external knowledge),
    formatted exactly in your required JSON format.
    """
    # Create input context
    outcomes = outcomes_module.get("primaryOutcomes", []) + outcomes_module.get("secondaryOutcomes", [])

    prompt = f"""
    You are a biomedical data analyst.
    
    You are given:
    - A clinical trial titled: "{trial_name}"
    - Studying the condition: "{condition}"
    - Using the investigational drug: "{drug}"
    - Here is the list of endpoints from this trial (measure, description, timepoint):
    
    ```json
    {json.dumps(outcomes, indent=2)}
    
    Your task is to search your knowledge and provide accurate endpoint **measurements** in the following JSON format.
    
    ⚠️ STRICT INSTRUCTIONS:
    - Only fill in measurement values that are well known or published from this or similar trials.
    - Do NOT guess or fabricate values. Leave fields as null if unknown.
    - If arms are not known specifically, use "intervention".
    - Do not invent statistical significance or confidence intervals unless clearly known.
    - If you cannot find data, leave fields empty or null.
    - Do not give any simple text in response. Only give the following JSON Array.
    
    Respond **only** in the following JSON format:
    
      [
        {{
          "name": "",
          "description": "",
          "timepoint": "",
          "arm": "intervention",
          "average_value": null,
          "upper_end": null,
          "lower_end": null,
          "statistical_significance": ""
        }}
      ]

      
        """
    
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000
        )
        print(response.choices[0].message.content.strip())
        return json.loads(response.choices[0].message.content.strip())

    except Exception as e:
        print(f"❌ Error calling GPT: {e}")
        return {"endpoints": []}


# Creating functions to Extract the trial info and format the trial output

In [62]:
from datetime import datetime, timedelta
import os
import json

def extract_baseline_characteristics(baseline_module):
    baseline = []
    group_list = baseline_module.get("groups", [])
    groups = {g["id"]: g.get("title", "Unnamed") for g in group_list}

    for measure in baseline_module.get("measures", []):
        title = measure.get("title", "N/A")
        unit = measure.get("unitOfMeasure", "")
        for cls in measure.get("classes", []):
            for cat in cls.get("categories", []):
                for meas in cat.get("measurements", []):
                    baseline.append({
                        "name": title,
                        "description": f"{title} - {cat.get('title', '')}".strip(" -"),
                        "timepoint": None,
                        "arm": groups.get(meas.get("groupId", ""), "N/A"),
                        "average_value": meas.get("value"),
                        "lower_end": meas.get("lowerLimit"),
                        "upper_end": meas.get("upperLimit"),
                        "statistical_significance": None,
                        "units": unit
                    })
    return baseline
    
def extract_endpoints(p):
    endpoints = []
    outcome_module = p["protocolSection"]["outcomesModule"]
    outcomes = outcome_module.get("primaryOutcomes", []) + outcome_module.get("secondaryOutcomes", [])

    for i, o in enumerate(outcomes):
        name = o.get("measure", "")
        description = o.get("description", "")
        timepoint = o.get("timeFrame", "")
        
        if p["hasResults"] == True:
            outcome_result = p["resultsSection"]["outcomeMeasuresModule"]["outcomeMeasures"][i]
            for j, g in enumerate(outcome_result.get("groups", [])):
                arm = g["title"]
                meas = outcome_result["classes"][0]["categories"][0]["measurements"][j]
                avg = meas.get("value", None)
                upper_end = meas.get("upperLimit", None)
                lower_end = meas.get("lowerLimit", None)

                stat_sig = ""
                for k in meas:
                    if k not in {"groupId", "id", "upperLimit", "lowerLimit", "value"}:
                        stat_sig += f"{k}: {meas[k]}; "

                endpoint = {
                    "name": name,
                    "description": description,
                    "timepoint": timepoint,
                    "arm": arm,
                    "average_value": avg,
                    "upper_end": upper_end,
                    "lower_end": lower_end,
                    "statistical_significance": stat_sig.strip()
                }
                endpoints.append(endpoint)
        else:
            endpoint = {
                "name": name,
                "description": description,
                "timepoint": timepoint,
                "arm": None,
                "average_value": None,
                "upper_end": None,
                "lower_end": None,
                "statistical_significance": None
            }
            endpoints.append(endpoint)

    return endpoints

            

def extract_trial_info(p):
    sponsor_info = p["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]
    study_t = p["protocolSection"]["designModule"]["studyType"]
    status = p["protocolSection"]["statusModule"]["overallStatus"]
    start_dt = p["protocolSection"]["statusModule"]["startDateStruct"]["date"]
    indication = p.get("protocolSection",{}).get("conditionsModule", {}).get("conditions", ["N/A"])
    
    trial = {
        "nct_id": p["protocolSection"]["identificationModule"]["nctId"],
        "title": p["protocolSection"]["identificationModule"]["briefTitle"],
        "indication": indication,
        "interventions": p.get("protocolSection",{}).get("armsInterventionsModule", {}).get("interventions", []),
        "phase": p.get("protocolSection",{}).get("designModule", {}).get("phases", ["N/A"])[0],
        "sponsor": sponsor_info["name"],
        "funder_type": sponsor_info.get("class", "N/A"),  # Extract funder type here
        "status": status,
        "study_type": study_t,
        "start_date": str(start_dt)
    }
        # Participants
    enrollment = p.get("protocolSection",{}).get("designModule", {}).get("enrollmentInfo", {})
    trial["participants"] = enrollment.get("count", "N/A")
    trial["protocolSection"] = p["protocolSection"]
    if p["hasResults"]:
        trial["resultsSection"] = p["resultsSection"]
    # Age Range (from eligibility info)
    eligibility = p.get("protocolSection",{}).get("eligibilityModule", {})
    trial["age_range"] = [
        eligibility.get("minimumAge", "N/A"),
        eligibility.get("maximumAge", "N/A")
    ]

    # Arms extraction
    arms_data = p.get("protocolSection",{}).get("armsInterventionsModule", {}).get("armGroups", [])
    arm_summary = {"intervention": 0, "placebo": 0}
    
    for arm in arms_data:
        arm_type = arm.get("type", "").lower()
        if "placebo" in arm_type or "comparator" in arm_type:
            arm_summary["placebo"] += 1
        elif "experimental" in arm_type or "intervention" in arm_type:
            arm_summary["intervention"] += 1
    
    trial["arms"] = arm_summary

    # Endpoints extraction
    outcomes = p.get("protocolSection",{}).get("outcomesModule", {})
    trial["primary_endpoints"] = [o["measure"] for o in outcomes.get("primaryOutcomes", [])]
    trial["secondary_endpoints"] = [o["measure"] for o in outcomes.get("secondaryOutcomes", [])]

    # Baseline Characteristics extraction
    if p.get("resultsSection",{}).get("baselineCharacteristicsModule"):
        baseline_module = p["resultsSection"]["baselineCharacteristicsModule"]
        trial["baseline_characteristics"] = extract_baseline_characteristics(baseline_module)


    # Extended structured endpoint extraction
    primary_outcomes_raw = outcomes.get("primaryOutcomes", [])
    secondary_outcomes_raw = outcomes.get("secondaryOutcomes", [])
    
    trial["structured_primary_endpoints"] = [
        {
            "name": o.get("measure"),
            "description": o.get("description", ""),
            "timepoint": o.get("timeFrame", "")
        }
        for o in primary_outcomes_raw
    ]
    
    trial["structured_secondary_endpoints"] = [
        {
            "name": o.get("measure"),
            "description": o.get("description", ""),
            "timepoint": o.get("timeFrame", "")
        }
        for o in secondary_outcomes_raw
    ]
    trial["hasResults"]= p["hasResults"]

    return trial
    
def format_trial_output(trial):

    
    return {
        "clinical_study": {
            "title": trial["title"],
            "nct_identifier": trial["nct_id"],
            "indication": ", ".join(trial["indication"]) if isinstance(trial["indication"], list) else trial["indication"],
            "intervention": ", ".join([trial["interventions"][i]["name"] for i in range(len(trial["interventions"]))]),
            "study_arms": trial.get("arms", {"intervention": 0, "placebo": 0}),
            "number_of_participants": trial.get("participants", 0),
            "average_age": trial.get("average_age"),
            "age_range": trial.get("age_range", ["N/A", "N/A"]),
            "interventional_drug": extract_drug_metadata_from_description(trial.get("interventions", [{}])),
            "endpoints": trial.get("primary_endpoints", []) + trial.get("secondary_endpoints", []),
            "baseline_characteristics": [b["description"] for b in trial.get("baseline_characteristics", [])]
        },
        "endpoints": extract_endpoints(trial) ,
        "baseline_measures": trial.get("baseline_characteristics", [])
    }


# Creating the trial output json and storing it as per the requirements

In [65]:
all_formatted_trials = []
for item in data:
    trial_data = item["trial"]
    trial = extract_trial_info(trial_data)
    if trial:
        formatted_trial = format_trial_output(trial)
        all_formatted_trials.append(formatted_trial)
        print(json.dumps(formatted_trial, indent=2))
        # Save to file
        trial_name = trial_data["protocolSection"]["sponsorCollaboratorsModule"]["leadSponsor"]["name"].replace(" ", "_").replace(",", "").replace(".", "")
        trial_folder = os.path.join("formatted_trials", trial_name)
        os.makedirs(trial_folder, exist_ok=True)
        file_path = os.path.join(trial_folder, f"{trial_data['protocolSection']['identificationModule']['nctId']}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(formatted_trial, f, ensure_ascii=False, indent=2)
        print(f"Saved formatted trial data for {trial_data['protocolSection']['identificationModule']['nctId']} to {file_path}")

    else:
        print("No valid trial data found.")

{
  "clinical_study": {
    "title": "A Study to Learn About the Study Medicine Called PF-07868489 in Healthy Adult People and in People With Pulmonary Arterial Hypertension",
    "nct_identifier": "NCT06137742",
    "indication": "Pulmonary Arterial Hypertension",
    "intervention": "PF-07868489, Placebo for PF-07868489",
    "study_arms": {
      "intervention": 1,
      "placebo": 1
    },
    "number_of_participants": 90,
    "average_age": null,
    "age_range": [
      "18 Years",
      "65 Years"
    ],
    "interventional_drug": [
      {
        "name": "PF-07868489",
        "dose": null,
        "frequency": null,
        "formulation": null
      },
      {
        "name": "Placebo for PF-07868489",
        "dose": null,
        "frequency": null,
        "formulation": "Placebo"
      }
    ],
    "endpoints": [
      "Number of Participants with Treatment Emergent Adverse Events (AEs) and Serious Adverse Events (SAEs)",
      "Number of Participants With Change From Base

# Ingesting the JSON into postgresSQL schema Tables: clinical_study, endpoints, baseline_measures

In [68]:
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

# PostgreSQL DB config — update with your actual credentials
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": os.getenv("dbname"),
    "user": os.getenv("user"),
    "password": os.getenv("password")
}
user = DB_CONFIG["user"]
password = DB_CONFIG["password"]
host = DB_CONFIG["host"]
port = DB_CONFIG["port"]
database = DB_CONFIG["dbname"]
# Create SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

def create_postgresql_schema():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # Create clinical_study table
    cur.execute("""
    DROP TABLE IF EXISTS clinical_study CASCADE;
    CREATE TABLE IF NOT EXISTS clinical_study (
        id SERIAL PRIMARY KEY,
        title TEXT,
        nct_identifier TEXT,
        indication TEXT,
        intervention TEXT,
        drug_name TEXT,
        drug_dose TEXT,
        drug_frequency TEXT,
        drug_formulation TEXT,
        arm_intervention INTEGER,
        arm_placebo INTEGER,
        number_of_participants INTEGER,
        average_age FLOAT,
        age_range TEXT
    );
    """)

    # Create endpoints table with foreign key to clinical_study
    cur.execute("""
    DROP TABLE IF EXISTS endpoints CASCADE;
    CREATE TABLE IF NOT EXISTS endpoints (
        id SERIAL PRIMARY KEY,
        study_id INTEGER NOT NULL,
        name TEXT,
        description TEXT,
        timepoint TEXT,
        arm TEXT,
        average_value FLOAT,
        upper_end FLOAT,
        lower_end FLOAT,
        statistical_significance TEXT,
        FOREIGN KEY (study_id) REFERENCES clinical_study(id) ON DELETE CASCADE
    );
    """)

    # Create baseline_measures table with foreign key to clinical_study
    cur.execute("""
    DROP TABLE IF EXISTS baseline_measures CASCADE;
    CREATE TABLE IF NOT EXISTS baseline_measures (
        id SERIAL PRIMARY KEY,
        study_id INTEGER NOT NULL,
        name TEXT,
        description TEXT,
        arm TEXT,
        average_value FLOAT,
        upper_end FLOAT,
        lower_end FLOAT,
        FOREIGN KEY (study_id) REFERENCES clinical_study(id) ON DELETE CASCADE
    );
    """)

    conn.commit()
    cur.close()
    conn.close()
    print("✅ Schema with foreign key dependencies created successfully.")

def insert_clinical_study(data):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    clinical = data["clinical_study"]

    # Insert into clinical_study
    cur.execute("""
    
        INSERT INTO clinical_study (
            title, nct_identifier, indication, intervention,
            drug_name, drug_dose, drug_frequency, drug_formulation,
            arm_intervention, arm_placebo,
            number_of_participants, average_age, age_range
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        RETURNING id;
    """, (
        clinical["title"],
        clinical["nct_identifier"],
        clinical["indication"],
        clinical["intervention"],
        ",".join(d["name"] if d.get("name") is not None else "NA" for d in clinical["interventional_drug"]),
        ",".join(d["dose"] if d.get("dose") is not None else "NA" for d in clinical["interventional_drug"]),
        ",".join(d["frequency"] if d.get("frequency") is not None else "NA" for d in clinical["interventional_drug"]),
        ",".join(d["formulation"] if d.get("formulation") is not None else "NA" for d in clinical["interventional_drug"]),
        clinical["study_arms"].get("intervention"),
        clinical["study_arms"].get("placebo"),
        clinical["number_of_participants"],
        clinical["average_age"],
        ", ".join([str(age) for age in clinical["age_range"] if age is not None])
    ))

    study_id = cur.fetchone()[0]

    # Insert endpoints
    for ep in data.get("endpoints", []):
        cur.execute("""
            INSERT INTO endpoints (
                study_id, name, description, timepoint, arm,
                average_value, upper_end, lower_end, statistical_significance
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, (
            study_id,
            ep.get("name"),
            ep.get("description"),
            ep.get("timepoint"),
            ep.get("arm"),
            ep.get("average_value"),
            ep.get("upper_end"),
            ep.get("lower_end"),
            ep.get("statistical_significance")
        ))

    # Insert baseline measures
    for bm in data.get("baseline_measures", []):
        cur.execute("""
            INSERT INTO baseline_measures (
                study_id, name, description, arm,
                average_value, upper_end, lower_end
            ) VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (
            study_id,
            bm.get("name"),
            bm.get("description"),
            bm.get("arm"),
            bm.get("average_value"),
            bm.get("upper_end"),
            bm.get("lower_end")
        ))

    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Inserted study ID: {study_id}")
def show_tables():
    
    # Connect to PostgreSQL
    conn = psycopg2.connect(**DB_CONFIG)
    
    # Create a cursor
    cur = conn.cursor()
    
    # Step 1: Get all table names from public schema
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' 
          AND table_type = 'BASE TABLE';
    """)
    
    tables = [row[0] for row in cur.fetchall()]
    print(f"📋 Found {len(tables)} tables.\n")
    
    # Step 2: For each table, print its name and contents
    for table in tables:
        print(f"🔹 Table: {table}")
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
        display(df) 
        print("\n" + "-"*60 + "\n")
    
    # Clean up
    cur.close()
    conn.close()

create_postgresql_schema()

✅ Schema with foreign key dependencies created successfully.


In [70]:
for trial in all_formatted_trials:
    insert_clinical_study(trial)
print("All trials inserted into PostgreSQL database.")

✅ Inserted study ID: 1
✅ Inserted study ID: 2
✅ Inserted study ID: 3
✅ Inserted study ID: 4
✅ Inserted study ID: 5
All trials inserted into PostgreSQL database.


In [72]:
print(len(data))

5


In [74]:
show_tables()

📋 Found 3 tables.

🔹 Table: clinical_study


/var/folders/6g/387qb0rn4_xdc_w69vs5mn9m0000gn/T/ipykernel_96333/1942569870.py:183: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table}", conn)


,id,title,nct_identifier,indication,intervention,drug_name,drug_dose,drug_frequency,drug_formulation,arm_intervention,arm_placebo,number_of_participants,average_age,age_range
0,1,A Study to Learn About the Study Medicine Call...,NCT06137742,Pulmonary Arterial Hypertension,"PF-07868489, Placebo for PF-07868489","PF-07868489,Placebo for PF-07868489","NA,NA","NA,NA","NA,Placebo",1,1,90,None,"18 Years, 65 Years"
1,2,A Study to Investigate the Safety and Efficacy...,NCT05975905,Pulmonary Arterial Hypertension,"Dose A KER-012, Dose B KER-012, Dose C KER-012...","Dose A KER-012,Dose B KER-012,Dose C KER-012,K...","NA,NA,NA,Dose B","Q4W,Q4W,Q4W,Q4W","NA,NA,NA,SC",3,1,113,None,"18 Years, N/A"
2,3,Inhaled Imatinib Pulmonary Arterial Hypertensi...,NCT05557942,Pulmonary Arterial Hypertension,AV-101,AV-101,NA,NA,dry powder inhalation,6,0,186,None,"18 Years, 75 Years"
3,4,A Study of a Mean Pulmonary Artery Pressure-Ta...,NCT05203510,Pulmonary Arterial Hypertension,"Parenteral Treprostinil, Oral Treprostinil","Parenteral Treprostinil,Oral Treprostinil","NA,NA","per schedule specified in the arm description,NA","Parenteral,Oral",1,0,52,None,"18 Years, N/A"
4,5,A Study of AV-101 (Dry Powder Inhaled Imatinib...,NCT05036135,Pulmonary Arterial Hypertension,"AV-101, Placebo","AV-101,Placebo","NA,NA","NA,NA","dry powder inhalation,dry powder inhalation",3,1,202,None,"18 Years, 75 Years"



------------------------------------------------------------

🔹 Table: endpoints


/var/folders/6g/387qb0rn4_xdc_w69vs5mn9m0000gn/T/ipykernel_96333/1942569870.py:183: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table}", conn)


,id,study_id,name,description,timepoint,arm,average_value,upper_end,lower_end,statistical_significance
0,1,1,Number of Participants with Treatment Emergent...,Part A,Baseline up to Day 113.,None,NaN,None,None,None
1,2,1,Number of Participants With Change From Baseli...,Part A,Baseline up to Day 113,None,NaN,None,None,None
2,3,1,Number of Participants With Vital Sign Abnorma...,Part A,Baseline up to Day 113,None,NaN,None,None,None
3,4,1,Number of Participants With Change From Baseli...,Part A,Baseline up to Day 113,None,NaN,None,None,None
4,5,1,Number of Participants with Treatment Emergent...,Part B,Baseline up to Day 253,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...
65,66,5,Phase 3: Change from Baseline in NT-proBNP,,24 weeks,None,NaN,None,None,None
66,67,5,Phase 3: Time to Clinical Worsening,,24 weeks,None,NaN,None,None,None
67,68,5,Phase 3: Proportion of Subjects with Improveme...,,24 weeks,None,NaN,None,None,None
68,69,5,Phase 3: Change from Baseline in Registry to E...,REVEAL Lite 2.0 risk scores range from 1 to 14...,24 weeks,None,NaN,None,None,None



------------------------------------------------------------

🔹 Table: baseline_measures


/var/folders/6g/387qb0rn4_xdc_w69vs5mn9m0000gn/T/ipykernel_96333/1942569870.py:183: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table}", conn)


,id,study_id,name,description,arm,average_value,upper_end,lower_end
0,1,3,"Age, Continuous","Age, Continuous",Placebo Crossover AV-101 10 mg,49.1,None,None
1,2,3,"Age, Continuous","Age, Continuous",Placebo Crossover AV-101 35 mg,45.9,None,None
2,3,3,"Age, Continuous","Age, Continuous",Placebo Crossover AV-101 70 mg,47.9,None,None
3,4,3,"Age, Continuous","Age, Continuous",Continuing AV-101 10 mg,45.5,None,None
4,5,3,"Age, Continuous","Age, Continuous",Continuing AV-101 35 mg,47.4,None,None
...,...,...,...,...,...,...,...,...
86,87,3,Race (NIH/OMB),Race (NIH/OMB) - Unknown or Not Reported,Placebo Crossover AV-101 70 mg,0.0,None,None
87,88,3,Race (NIH/OMB),Race (NIH/OMB) - Unknown or Not Reported,Continuing AV-101 10 mg,5.0,None,None
88,89,3,Race (NIH/OMB),Race (NIH/OMB) - Unknown or Not Reported,Continuing AV-101 35 mg,4.0,None,None
89,90,3,Race (NIH/OMB),Race (NIH/OMB) - Unknown or Not Reported,Continuing AV-101 70 mg,3.0,None,None



------------------------------------------------------------



In [ ]:
import os
import requests
from google_search_results import GoogleSearchResults

SERP_API_KEY = os.getenv("SERPAPI_API_KEY")

def search_and_download_papers(trial_title, trial_nctid, num_results=5):
    if not SERP_API_KEY:
        raise ValueError("Missing SERPAPI_API_KEY in environment variables")
    params = {
        "engine": "google",
        "q": f"{trial_title} {trial_nctid} clinical trial pdf",
        "num": num_results,
        "api_key": SERP_API_KEY
    }
    search = GoogleSearchResults(params)
    results = search.get_dict()
    papers_folder = os.path.join("research_papers", trial_nctid)
    os.makedirs(papers_folder, exist_ok=True)
    found = 0
    for res in results.get("organic_results", []):
        link = res.get("link", "")
        if link.endswith(".pdf"):
            try:
                response = requests.get(link, timeout=15)
                if response.status_code == 200:
                    filename = os.path.join(papers_folder, f"paper_{found+1}.pdf")
                    with open(filename, "wb") as f:
                        f.write(response.content)
                    print(f"Downloaded: {filename}")
                    found += 1
                if found >= num_results:
                    break
            except Exception as e:
                print(f"Failed to download {link}: {e}")

# Example usage for your filtered trials:
for t in filtered:
    title = t["protocolSection"]["identificationModule"]["briefTitle"]
    nctid = t["protocolSection"]["identificationModule"]["nctId"]
    search_and_download_papers(title, nctid, num_results=3)

ImportError: cannot import name 'GoogleSearch' from 'serpapi' (/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/serpapi/__init__.py)

In [15]:
%pip install google-search-results


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
